In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from datasets import MultivariateNormalDataset
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_gpus = 1 if device=='cuda' else 0
print(device)

cuda


In [ ]:
%load_ext autoreload
%autoreload 2
# from models import mine
from models.mine import MutualInformationEstimator
from pytorch_lightning import Trainer
import logging
logging.getLogger().setLevel(logging.ERROR)

dim = 1
N = 3000
lr = 1e-4
epochs = 200
batch_size = 500

steps = 15
rhos = np.linspace(-0.99, 0.99, steps)
loss_type = ['mine_biased']

results_dict = dict()

for loss in loss_type:
    results = []
    for rho in rhos:
        train_loader = torch.utils.data.DataLoader(
        MultivariateNormalDataset(N, dim, rho), batch_size=batch_size, shuffle=True)
        test_loader = torch.utils.data.DataLoader(
        MultivariateNormalDataset(N, dim, rho), batch_size=batch_size, shuffle=True)

        
        true_mi = train_loader.dataset.true_mi

        kwargs = {
            'lr': lr,
            'batch_size': batch_size,
            'train_loader': train_loader,
            'test_loader': test_loader,
            'alpha': 1.0
        }

        model = MutualInformationEstimator(
            dim, dim, loss=loss, **kwargs).to(device)
        
        trainer = Trainer(max_epochs=epochs, early_stop_callback=False, gpus=num_gpus)
        trainer.fit(model)
        trainer.test()

        print("True_mi {}".format(true_mi))
        print("MINE {}".format(model.avg_test_mi))
        results.append((rho, model.avg_test_mi, true_mi))

    results = np.array(results)
    results_dict[loss] = results

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'mine'